In [1]:
import os
import chess
import torch
import numpy as np
import pandas as pd
DATA_FOLDER = os.path.join("..", "data", "batches")

In [3]:
batch = pd.read_csv(os.path.join(DATA_FOLDER, "batch_28.csv"))

In [4]:
features = 12 * 64 + 1

FEATURE_TEMPO =  12  * 64

feature_matrix = np.zeros((len(batch), features), dtype=np.int8)
labels = np.array(batch.iloc[:, -1], dtype=np.float32)

In [5]:
from chess import PAWN, KNIGHT, BISHOP, ROOK, QUEEN, KING, WHITE, BLACK, Piece
W_PAWN, W_KNIGHT, W_BISHOP, W_ROOK, W_QUEEN, W_KING = Piece(PAWN, WHITE), Piece(KNIGHT, WHITE), Piece(BISHOP, WHITE), Piece(ROOK, WHITE), Piece(QUEEN, WHITE), Piece(KING, WHITE)
B_PAWN, B_KNIGHT, B_BISHOP, B_ROOK, B_QUEEN, B_KING = Piece(PAWN, BLACK), Piece(KNIGHT, BLACK), Piece(BISHOP, BLACK), Piece(ROOK, BLACK), Piece(QUEEN, BLACK), Piece(KING, BLACK)
semantical_piece_order = [W_PAWN, W_KNIGHT, W_BISHOP, W_ROOK, W_QUEEN, W_KING,
                          B_PAWN, B_KNIGHT, B_BISHOP, B_ROOK, B_QUEEN, B_KING]
piece_feature_map = { piece: index * 64 for (index, piece) in enumerate(iter(semantical_piece_order))}


#Piece Index map is how pieces are defined in Scam, see https://github.com/fabianvdW/Scam/blob/81f8f85bc4f52655b852f87be43546c7dfea6c8c/src/types.rs#L72-L84
piece_index_map = {W_PAWN: 1, W_KNIGHT: 2 , W_BISHOP: 3 , W_ROOK: 4 , W_QUEEN: 5 , W_KING: 6 ,
                   B_PAWN: 9, B_KNIGHT: 10, B_BISHOP: 11, B_ROOK: 12, B_QUEEN: 13, B_KING: 14}
piece_max_index = 15

piece_values = {W_PAWN:  100/512, W_KNIGHT:  325/512, W_BISHOP:  350/512, W_ROOK:  550/512, W_QUEEN:  1000/512, W_KING: 0,
                B_PAWN: -100/512, B_KNIGHT: -325/512, B_BISHOP: -350/512, B_ROOK: -550/512, B_QUEEN: -1000/512, B_KING: 0}

def initialize_piece_values(model):
    weights = model.state_dict()['1.weight']
    for piece in semantical_piece_order:
        for sq in range(64):
            weights[0, piece_feature_map[piece] + sq] += piece_values[piece]

In [6]:
def fill_features(matrix, at_index, fen):
    tokens = fen.split(" ")
    epd, tempo = tokens[0], int({"w": 1, "b": -1}[tokens[1]])
    matrix[at_index, FEATURE_TEMPO] = tempo
    board = chess.BaseBoard(board_fen = epd)
    for piece in semantical_piece_order:
        for sq in board.pieces(piece.piece_type, piece.color):
            matrix[at_index, piece_feature_map[piece] + sq] = 1

def get_features(fen):
    res = np.zeros((1, features), dtype=np.int8)
    fill_features(res, 0, fen)
    return res
def apply_model(model, fen):
    return model(torch.tensor(get_features(fen)))

In [7]:
for i in range(len(batch)):
    fill_features(feature_matrix, i, batch.iloc[i, 0])
np.save(os.path.join(DATA_FOLDER,"feature_matrix_batch_28"), feature_matrix)

In [8]:
#Shuffling the data, making a train validation split
perm = np.random.permutation(len(feature_matrix))
feature_matrix = feature_matrix[perm]
labels = labels[perm]
N = int(0.9 * len(feature_matrix))
x_train, x_val, y_train, y_val = feature_matrix[:N], feature_matrix[N:], labels[:N], labels[N:]
y_train, y_val = np.expand_dims(y_train, axis = 1), np.expand_dims(y_val, axis = 1)
x_train, y_train, x_val, y_val = map(torch.tensor, (x_train, y_train, x_val, y_val))

In [9]:
from torch import optim,nn
from torch.utils.data import TensorDataset, DataLoader
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size = 64, shuffle=True)
val_ds = TensorDataset(x_val, y_val)
val_dl = DataLoader(val_ds, batch_size = 64)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    def forward(self, x):
        return self.lambd(x)
    
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item()

def fit(model, epochs, loss_func, opt, train_dl, val_dl):
    for epoch in range(epochs):
        model.train()
        
        train_loss = 0
        for xb, yb in train_dl:
            train_loss += loss_batch(model, loss_func, xb, yb, opt)
        train_loss /= len(train_ds)
        
        model.eval()
        with torch.no_grad():
            val_loss = sum(loss_func(model(xb), yb) for xb, yb in val_dl)
            val_loss /= len(val_ds)
        print("Finished epoch {}".format(epoch))
        print("Train loss: {}, Val loss: {}".format(train_loss, val_loss))
        
def get_model():
    model = nn.Sequential(
        LambdaLayer(lambda x: x.type(torch.FloatTensor)),
        nn.Linear(features, 1),
        nn.Sigmoid()
    )
    opt = optim.Adam(model.parameters())
    loss_func = nn.MSELoss(reduction='sum')
    return model, opt, loss_func
model, opt, loss_func = get_model()
initialize_piece_values(model)
fit(model, 30, loss_func, opt, train_dl, val_dl)

Finished epoch 0
Train loss: 0.09594909528043535, Val loss: 0.09461022168397903
Finished epoch 1
Train loss: 0.09459220668501324, Val loss: 0.09428969770669937
Finished epoch 2
Train loss: 0.09419109808312522, Val loss: 0.09382405132055283
Finished epoch 3
Train loss: 0.09389035350693596, Val loss: 0.09360276162624359
Finished epoch 4
Train loss: 0.09361272689183553, Val loss: 0.0932682678103447
Finished epoch 5
Train loss: 0.0933847198735343, Val loss: 0.09304667264223099
Finished epoch 6
Train loss: 0.09316931249247656, Val loss: 0.09282287210226059
Finished epoch 7
Train loss: 0.09296949906720056, Val loss: 0.09267483651638031
Finished epoch 8
Train loss: 0.09279771267573039, Val loss: 0.09254957735538483
Finished epoch 9
Train loss: 0.09262937711689208, Val loss: 0.09235011041164398
Finished epoch 10
Train loss: 0.09246484813637204, Val loss: 0.09223499894142151
Finished epoch 11
Train loss: 0.09230691453907225, Val loss: 0.09205810725688934
Finished epoch 12
Train loss: 0.09217182

In [10]:
torch.save(model.state_dict(), os.path.join(DATA_FOLDER,"batch_28model.pkl"))
model.load_state_dict(torch.load(os.path.join(DATA_FOLDER, "batch_28model.pkl")))

<All keys matched successfully>

In [11]:
params = list(model.parameters())
weights = params[0].detach().numpy()[0]
bias = params[1].detach().numpy()[0]
sc = 2**17
shift = 2**9
def scale(w):
    return round(sc * w)
def print_psqt(weights):
    res = np.zeros((15, 64))
    for piece in semantical_piece_order:
        for sq in range(64):
            res[piece_index_map[piece], sq] = scale(weights[piece_feature_map[piece] + sq])
    res_str = "["
    for i in range(len(res)):
        res_str += "["
        for j in range(len(res[i])):
            res_str += str(int(res[i, j])) +", "
        res_str += "], "
    res_str += "]"
    return res_str
rust_psqt = "pub const PSQT: [[i32; 64]; {}] = {};".format(piece_max_index, print_psqt(weights[:-1]))
rust_tempo_bonus = "pub const TEMPO_BONUS: i32 = {};".format(scale(weights[-1]))
rust_bias = "pub const BIAS: i32 = {};".format(scale(bias))
rust_shift = "pub const DIV: i32 = {};".format(shift)
print(rust_psqt)
print(rust_tempo_bonus)
print(rust_bias)
print(rust_shift)

pub const PSQT: [[i32; 64]; 15] = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ], [26718, 29210, 22080, 25876, 26512, 21141, 26715, 29763, 66162, 69152, 60572, 63500, 63342, 74964, 82719, 61413, 66074, 65381, 58556, 62142, 65883, 68734, 73925, 64307, 70812, 75164, 62970, 63477, 67658, 70119, 76045, 65689, 83038, 84535, 76455, 76180, 78969, 77112, 83845, 67121, 108256, 119883, 121712, 110698, 109552, 108923, 115469, 103648, 173170, 170701, 158304, 145108, 141844, 122494, 125341, 133110, 30067, 26977, 29039, 29656, 22731, 27128, 28616, 28981, ], [115159, 153706, 148022, 158503, 160105, 163225, 148642, 76445, 148303, 168633, 171830, 170477, 172303, 170481, 162843, 162885, 157917, 178394, 176239, 189140, 193126, 185174, 184740, 165133, 165899, 192339, 194880, 197140, 199225, 197454, 199805, 182664, 176509, 195509, 202675, 212549, 208066, 21911